In [4]:
import networkx as nx
import pandas
import pylab as plt  #import Matplotlib plotting interface

df = pandas.read_csv("sin_dataset.csv")
df = df.set_index("Countries")
f = open('myfile.adjlist', 'w')
for i in range(0,20):
    #write row country in same line
    row = df.index[i]
    f.write(row)
    for j in range(0,20):
        if df.iloc[i,j] == 1:
            #write column country in same line
            col = df.columns[j]
            f.write(" "+col)
    #new line
    if(i != 19):
        f.write("\n")
f.close()
fh=open("myfile.adjlist", 'rb')
G=nx.read_adjlist(fh)

def draw_graph():
    graph_pos = nx.spring_layout(G)
    # draw nodes, edges and labels
    nx.draw_networkx_nodes(G, graph_pos, node_size=250  , node_color='blue', alpha=0.15)
    nx.draw_networkx_edges(G, graph_pos, edge_color='red')
    nx.draw_networkx_labels(G, graph_pos, font_size=10, font_family='sans-serif', font_color='black')
    plt.savefig("graph.png")
    # show graph
    #plt.show()
draw_graph()